# Performance by product --> SoM Dashboard

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/Brands/SoM/data/'
CPG = 'danone' 
nielsen_cat = True #True JUST FOR RB_HYHO, DANONE, UNILEVER SO FAR!!!
past = False
SOM_CPG = CPG.title()

## Import

In [2]:
import os
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
print([x for x in os.listdir(BASE_DIR) if x.endswith(f'{CPG}.csv')])

['som_danone.csv', 'nielsen_danone.csv', 'products_danone.csv', 'nielsen_pred_danone.csv']


In [3]:
df = pd.read_csv(os.path.join(BASE_DIR,f'products_{CPG}.csv'),
                 sep='\t', encoding='UTF-16', low_memory=False)
df.sample(2)

,month,country,city name,store_name,category en,category_id,product id,product name,barcodes,package,...,sales,orders,avg ticket,users,units found,units replaced,units requested,found rate,fulfillment,frequency
13319,septiembre 2021,MX,Puebla,La Comer,Yogurt,24,336249,Yoghurt bebible doble cero con durazno,{7501040091261},242 g,...,34,1,34,1,3,0,3,100.0%,100.0%,1.00
1462,septiembre 2021,MX,Chihuahua,Alsuper Plus,Desserts & Refrigerated Bakery,22,337122,Arroz con leche,{7501020540628},4 x 125 g,...,896,30,30,29,36,0,36,100.0%,100.0%,1.03


In [4]:
# df.rename(columns={'month  / year':'month  ','sales [currency]':'sales'}, inplace=True)

In [5]:
aux_ini = df.copy()
aux_ini['sales'] = aux_ini['sales'].astype(str).str.replace(',','').astype(float)
aux_ini = aux_ini.groupby('month  ')['sales'].sum().reset_index()
aux_ini

,month,sales
0,septiembre 2021,17280317.00


In [6]:
import re
nielsen = pd.read_csv(BASE_DIR + f'nielsen_{CPG}.csv',
                      sep='\t',encoding='utf-16'
                     ).drop_duplicates(subset=['UPC','PRODUCT ID'])
display(nielsen['UPC'].head())
nielsen['UPC'] = nielsen['UPC'].apply(lambda x: re.sub('[^\d]','',str(x)))
nielsen['UPC'] = nielsen['UPC'].astype(str).str.strip().str.split().str[0].fillna(0).astype(int)
display(nielsen['UPC'].head())
display(nielsen['UPC'].head())
nielsen.sample(2)

0    75009076
1    75009083
2    75009090
3    75010065
4    75010751
Name: UPC, dtype: object

0    75009076
1    75009083
2    75009090
3    75010065
4    75010751
Name: UPC, dtype: int64

0    75009076
1    75009083
2    75009090
3    75010065
4    75010751
Name: UPC, dtype: int64

,UPC,FORMATO,PROVEEDOR GENERAL,PROVEEDOR ABIERTO,MARCA DETALLE,MARCA GENERICA,GRAMAJE,DESCRIPCION NIELSEN,PRODUCT ID,SEGMENTO,SUBSEGMENTO 1,SUBSEGMENTO 2,SUBSEGMENTO 3
4831,400095797937,GARR,COCA COLA,COCA COLA,CIEL,CIEL,20.00,CIEL AGUA NATURAL NATURAL 20 LT GARR,NaN,AGUA NATURAL,AGUA NATURAL,NaN,NaN
3549,7075140005027,IND,DEMAS FABRICANTES,DEMAS FABRICANTES,CRYSTAL GEYSER,CRYSTAL GEYSER,0.50,AGUA CRYSTAL GEYSER ALPINE 500 ML 500 M,NaN,AGUA NATURAL,AGUA PREMIUM,NaN,NaN


## Functions

In [7]:
import time
import numpy as np
from IPython.lib.display import Audio

start = time.time()
def time_exp(x):
    minutes, seconds = np.floor(x/60), 60*(x/60-np.floor(x/60))
    print(f"{'{:.0f}'.format(minutes)} minutos con {'{:.2f}'.format(seconds)} segundos")
    
def tono(a = 1000, b = 700, play_time_seconds = 1, framerate = 4410):
    t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)*np.pi
    return Audio(np.sin(a*t) + np.sin(b*t), rate = framerate, autoplay=True)

In [8]:
def limpio(data,month_col,to_drop):
    df = data.copy()
    df[['mes','anio']] = df[month_col].str.split(expand=True)
    meses = dict(zip(['enero','febrero','marzo','abril','mayo','junio','julio',
                      'agosto','septiembre','octubre','noviembre','diciembre'],range(1,13)))
    df['mes'] = df['mes'].replace(meses)
    df.drop(columns=to_drop,inplace=True)
    df.columns = [str(x).replace('.','').replace(' ','_').lower() for x in df.columns]
    for col in ['brand_filter','cpg_filter']:
        df[f'{col[:-7]}_id'] = df[col].apply(lambda x: ','.join(re.findall('\s\((\d+)\)',str(x))))
        df[f'{col[:-7]}_name'] = df[col].apply(lambda x: re.sub('\s\(\d+\)','',str(x)))
    return df

## Transform

### Barcodes

In [9]:
df['barcodes'] = df['barcodes'].str.replace('{','').str.replace('}','')
df['barcodes'] = df['barcodes'].str.split(',',n=1).fillna('.').apply(lambda x:x[0])
df['barcodes'].sample(4)

/var/folders/j4/1p33_bh96yn8pdz3b_5t80hm0000gn/T/ipykernel_4594/2845833164.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['barcodes'] = df['barcodes'].str.replace('{','').str.replace('}','')


8168       25293003651
2452     7502252481666
12785     818290010391
14806    7501055363346
Name: barcodes, dtype: object

### Merge

In [10]:
df = df.astype(str).merge(nielsen.astype(str).rename(columns={'UPC':'barcodes',
                                                              'PRODUCT ID':'product id'}),
                          on=['barcodes'] if nielsen_cat else ['barcodes','product id'],
                          how='left' if nielsen_cat else 'left')
if nielsen_cat:
    df = df.rename(columns={'product id_x':'product_id'}).drop(columns='product id_y')

df.sample(2)

,month,country,city name,store_name,category en,category_id,product_id,product name,barcodes,package,...,PROVEEDOR GENERAL,PROVEEDOR ABIERTO,MARCA DETALLE,MARCA GENERICA,GRAMAJE,DESCRIPCION NIELSEN,SEGMENTO,SUBSEGMENTO 1,SUBSEGMENTO 2,SUBSEGMENTO 3
16397,septiembre 2021,MX,Tijuana,Soriana,Yogurt,24,335504,Yoghurt doble cero natural,7501040090394,1 kg,...,SIGMA,SIGMA,YOPLAIT DOBLE CERO,YOPLAIT DOBLE CERO,1.0,YOPLAIT BATIDO NATURAL LIGHT 0% GRASA 1000GR. ...,LIGHT,LIGHT,nan,nan
4030,septiembre 2021,MX,Ciudad de México,Fresko,Yogurt,24,1256963,Yogurt batido con limón,7501055368440,125 g,...,SANTA CLARA,SANTA CLARA,SANTA CLARA,SANTA CLARA,0.125,SANTA CLARA YOGHURT SABOR LIMON SOLIDO VASO 12...,BB SOLIDO,BASE BUSINESS,nan,nan


### CPG

In [11]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('month  ')['sales'].sum().reset_index()
aux = aux_ini.copy()
aux = aux.set_index('month  ').join(aux_fin.set_index('month  '),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
septiembre 2021,17280317.00,17280317.00,0.00,0.00


In [12]:
prod_cpg = pd.read_csv(os.path.join(BASE_DIR,'pbbc.csv'),
                       #sep='\t',encoding='utf-16'
                      )
prod_cpg.sample(4)

,n,product_id,barcodes,category_id,category_name,brand_id,brand_name,cpg_id,cpg_name
93823,1,2019393,40000656971,327,Tortas y pasteles,16792,Kirkland Signature,1089,Kirkland
11006,1,131139,11209003662,931,Salsas y aderezos,8701,Morehouse,781,Smart
53845,1,353409,96619107698,1004,"Nueces, semillas y frutas secas",16792,Kirkland Signature,1089,Kirkland
15342,1,151734,40000505645,324,Pan rústico,16792,Kirkland Signature,1696,Imalinx


In [13]:
prod_cpg.drop_duplicates(('barcodes' if nielsen_cat else 'product_id'),inplace=True)
prod_cpg = prod_cpg.astype(str).rename(columns={'product_id':'product id'})
prod_cpg['cpg filter'] = prod_cpg['cpg_name']+" ("+prod_cpg['cpg_id'].astype(str)+")"
prod_cpg.sample(4)

,n,product id,barcodes,category_id,category_name,brand_id,brand_name,cpg_id,cpg_name,cpg filter
59556,1,383078,810990005738,35,Agua,4067,Calahua,343,Calahua,Calahua (343)
264,1,90318,7501032908720,928,Almacenamiento de cocina,662,Ziploc,10,SC Johnson,SC Johnson (10)
99607,1,2355690,3501179314969,277,Escritura,18041,Parker,975,Newell Brands,Newell Brands (975)
62392,1,439599,7501058639080,244,Electrodomésticos,65,Nescafé Dolce Gusto,3,Nestlé MX,Nestlé MX (3)


In [14]:
df = df.merge(prod_cpg[[('barcodes' if nielsen_cat else 'product id'),'cpg filter']],
              on=('barcodes' if nielsen_cat else 'product id'),how='left')

In [15]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('month  ')['sales'].sum().reset_index()
aux = aux_ini.copy()
aux = aux.set_index('month  ').join(aux_fin.set_index('month  '),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
septiembre 2021,17280317.00,17280317.00,0.00,0.00


### Limpio

In [16]:
df = limpio(df,'month  ',
            ['month  ','country ','avg ticket','found rate','fulfillment','frequency'])
df.sample(2)

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_filter,...,subsegmento_1,subsegmento_2,subsegmento_3,cpg_filter,mes,anio,brand_id,brand_name,cpg_id,cpg_name
7278,Guadalajara,Chedraui,Yogurt,24,336476,Yoghurt doble cero sabor fresa,7501040090806,125 g,nan,Yoplait (6413),...,LIGHT,nan,nan,Sigma Total (72),9,2021,6413,Yoplait,72,Sigma Total
17359,Toluca,La Comer,Yogurt,24,3792813,Yoghurt griego blueberry,818290017659,150 g,nan,Chobani (14849),...,NaN,NaN,NaN,Chobani (284),9,2021,14849,Chobani,284,Chobani


In [17]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))
aux = aux_ini.copy()
aux['month  '] = aux['month  '].str.split().str[0].str[:3]
aux = aux.set_index('month  ').join(aux_fin.set_index('mes'),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
sep,17280317.00,17280317.00,0.00,0.00


In [18]:
aux = df[df['formato'].astype(str)=='nan'].copy()
df = df[~(df['formato'].astype(str)=='nan')].copy()
null_fill = {'proveedor_general':'cpg_name',
             'proveedor_abierto':'cpg_name',
             'marca_detalle':'brand_name',
             'marca_generica':'brand_name',
             'gramaje':'package',
             'descripcion_nielsen':'product_name',
             'segmento':'category_en'}
for x,y in null_fill.items():
    aux.loc[:,x] = aux.loc[:,y]
    
df = df.append(aux,ignore_index=True)

In [19]:
for col in df.columns:
    df[col] = df[col].astype(str).str.replace('nan','')
    
df = df.drop(columns='cpg_name').fillna('')
df.sample()

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_filter,...,segmento,subsegmento_1,subsegmento_2,subsegmento_3,cpg_filter,mes,anio,brand_id,brand_name,cpg_id
10570,Puebla,Chedraui,Plant-based Beverages,1516,344211,Bebida a base de soya sabor vainilla,7501005117777,946 ml,,AdeS (38),...,BEBIDAS SABORIZADAS,BEBIDAS REFRESCANTES,,,Coca Cola (52),9,2021,38,AdeS,52


In [20]:
brand_cols = ['formato','proveedor_general','proveedor_abierto','marca_detalle','marca_generica','gramaje','descripcion_nielsen','segmento','subsegmento_1','subsegmento_2','subsegmento_3']
display(df[brand_cols].sample())

for col in brand_cols:
    df[col] = df[col].str.strip().str.title()

display(df[brand_cols].sample())

,formato,proveedor_general,proveedor_abierto,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3
8862,INDIVIDUAL,ALPURA,ALPURA,ALPURA,ALPURA,0.25,ALPURA P/BEBER FRESA 250GR NAL,BB LIQUIDO,BASE BUSINESS,,


,formato,proveedor_general,proveedor_abierto,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3
1618,Ind,Danone,Danone,Juizzy,Juizzy,1.0,Ag Sab Guaya Sabor Con Azuc Sin I Juizzy Guaya...,Bebidas Saborizadas,Agua Saborizada,,


In [21]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))
aux_fin

,mes,sales
0,sep,17280317.00


### Marca genérica

In [22]:
df.loc[df[df['marca_generica']==''].index, 'marca_generica'] = df.loc[df[df['marca_generica']==''].index, 'marca_detalle']
df[df['marca_generica']=='']

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_filter,...,segmento,subsegmento_1,subsegmento_2,subsegmento_3,cpg_filter,mes,anio,brand_id,brand_name,cpg_id


## Export

### Same product_id as past

In [23]:
print(len(df))
if past:
    som = pd.ExcelFile(os.path.join('/'.join(BASE_DIR.split('/')[:-2]),
                                    f'SoM_{SOM_CPG}.xlsx')).parse(sheet_name=0)
    df = df.merge(som.drop_duplicates('product_id')[['product_id']].reset_index(drop=True).astype(str))
    
print(len(df))

17606
17606


In [24]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))

In [25]:
aux = aux_ini.copy()
aux['month  '] = aux['month  '].str.split().str[0].str[:3]
aux = aux.set_index('month  ').join(aux_fin.set_index('mes'),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1

### Ordered columns

In [26]:
export = df[['city_name', 'store_name', 'category_en', 'category_id', 'product_id',
             'product_name', 'barcodes', 'package', 'product_sku', 'brand_name',
             'sales', 'orders', 'users', 'units_found', 'units_replaced',
             'units_requested', 'formato', 'proveedor_general', 'proveedor_abierto',
             'marca_detalle', 'marca_generica', 'gramaje', 'descripcion_nielsen',
             'segmento', 'subsegmento_1', 'subsegmento_2', 'subsegmento_3',
             'mes', 'anio']].copy()
export.sort_values(['anio','mes'],inplace=True)
export.sample(4)

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_name,...,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3,mes,anio
5076,Ciudad de México,Sumesa,Yogurt,24,336057,Yogurt bebible con mango,7501055361212,235 g,,Santa Clara,...,Santa Clara,Santa Clara,0.235,Santa Clara Bebible Mango 235Gr,Bb Liquido,Base Business,,,9,2021
3744,Ciudad de México,La Comer,Plant-based Beverages,1516,869057,Leche de coco,8850781700055,400 ml,,Thai Heritage,...,Thai,Thai,0.4,Thai Heritage Leche Sabor Coco Lata 400Ml 8850...,Plant Based,Plant Based,Coco,Lata,9,2021
11737,Querétaro,Chedraui,Water,35,339040,Agua natural,7501022009352,6 x 1 l,,Epura,...,E Pura,E Pura,1.0,E-Pura Agua Natural 6Pack Bot Plast 1L C/U 6000Ml,Agua Natural,Agua Natural,,,9,2021
12652,Querétaro,HEB,Plant-based Beverages,1516,335002,Alimento líquido 3 nueces,7501020552935,Cartón 960 ml,,Lala Vita,...,Lala Soy Vita,Lala Soy Vita,0.96,Lala Soy Vita 3 Nueces Carton 960Ml Nal 750102...,Plant Based,Plant Based,Nuez,Uht,9,2021


In [27]:
export.to_csv(os.path.join(BASE_DIR,f'som_{CPG}.csv'),index=False,sep='\t',encoding='utf-16')

## Fin

In [28]:
aux

,sales,sales_fin,dif,dif_p
month,,,,
sep,17280317.00,17280317.00,0.00,0.00


In [29]:
time_exp(time.time() - start)
tono()

0 minutos con 3.11 segundos
